In [1]:
import numpy as np
import pandas as pd
import matplotlib. pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler

from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential

In [2]:
df=pd.read_csv(r"C:\Users\Priyanshul\OneDrive\Desktop\Hackathon\stock-market-prediction-and-sentimental-analysis\DJIA_table(train).csv",index_col='Date',parse_dates=True,infer_datetime_format=True)

In [3]:
df=df.iloc[::-1]
df.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2008-08-08,11432.08984,11759.95996,11388.04004,11734.32031,212830000,11734.32031
2008-08-11,11729.66992,11867.11035,11675.53027,11782.34961,183190000,11782.34961
2008-08-12,11781.70020,11782.34961,11601.51953,11642.46973,173590000,11642.46973
2008-08-13,11632.80957,11633.78027,11453.33984,11532.95996,182550000,11532.95996
2008-08-14,11532.07031,11718.28027,11450.88965,11615.92969,159790000,11615.92969


In [4]:
target_y=df['Close']
x_feat=df.iloc[:,0:3]

In [5]:
target_y

Date
2008-08-08    11734.32031
2008-08-11    11782.34961
2008-08-12    11642.46973
2008-08-13    11532.95996
2008-08-14    11615.92969
                 ...     
2015-12-24    17552.16992
2015-12-28    17528.26953
2015-12-29    17720.98047
2015-12-30    17603.86914
2015-12-31    17425.02930
Name: Close, Length: 1863, dtype: float64

In [6]:
x_feat

,Open,High,Low
Date,,,
2008-08-08,11432.08984,11759.95996,11388.04004
2008-08-11,11729.66992,11867.11035,11675.53027
2008-08-12,11781.70020,11782.34961,11601.51953
2008-08-13,11632.80957,11633.78027,11453.33984
2008-08-14,11532.07031,11718.28027,11450.88965
...,...,...,...
2015-12-24,17593.25977,17606.33984,17543.94922
2015-12-28,17535.66016,17536.90039,17437.33984
2015-12-29,17547.36914,17750.01953,17547.36914


In [14]:
sc = MinMaxScaler()

In [15]:
x_ft = sc.fit_transform(x_feat.values)
x_ft = pd.DataFrame(columns=x_feat.columns,data=x_ft,index=x_feat.index)

In [16]:
x_ft.head()

,Open,High,Low
Date,,,
2008-08-08,0.415114,0.433814,0.416695
2008-08-11,0.440401,0.443018,0.441053
2008-08-12,0.444822,0.435737,0.434783
2008-08-13,0.432170,0.422975,0.422228
2008-08-14,0.423610,0.430233,0.422020


In [17]:
def lstm_split(data, n_steps):
    x,y= [],[]
    for i in range(len(data)-n_steps+1):
        x.append(data[i:i + n_steps, :1])
        y.append(data[i + n_steps-1, :1])
    return np.array(x),np.array(y)

In [18]:
x1,y1 = lstm_split(x_ft.values,n_steps=2)
train_split=0.8
split_idx = int(np.ceil(len(x1)*train_split))
date_index = x_ft.index

x_train,x_test = x1[:split_idx],x1[split_idx:]
y_train,y_test = y1[:split_idx],y1[split_idx:]

In [19]:
x_train

array([[[0.41511378],
        [0.4404009 ]],

       [[0.4404009 ],
        [0.44482222]],

       [[0.44482222],
        [0.43217011]],

       ...,

       [[0.89012873],
        [0.88118419]],

       [[0.88118419],
        [0.88658182]],

       [[0.88658182],
        [0.88131   ]]])

In [20]:
x_test.shape

(372, 2, 1)

In [21]:
y_train.shape

(1490, 1)

In [22]:
y_test.shape

(372, 1)

In [48]:
lstm= Sequential()
lstm.add(LSTM(32,input_shape=(x_train.shape[1],x_train.shape[2]),activation='relu',return_sequences=False))
lstm.add(Dense(1))

In [24]:
lstm.compile(loss='mean_squared_error',optimizer='adam')

In [25]:
history=lstm.fit(x_train,y_train,epochs=20,batch_size=4,verbose=2,shuffle=False)

Epoch 1/20
373/373 - 4s - loss: 0.0094 - 4s/epoch - 11ms/step
Epoch 2/20
373/373 - 1s - loss: 0.0020 - 1s/epoch - 4ms/step
Epoch 3/20
373/373 - 1s - loss: 0.0018 - 1s/epoch - 3ms/step
Epoch 4/20
373/373 - 1s - loss: 0.0015 - 1s/epoch - 3ms/step
Epoch 5/20
373/373 - 1s - loss: 0.0013 - 1s/epoch - 3ms/step
Epoch 6/20
373/373 - 1s - loss: 0.0011 - 1s/epoch - 4ms/step
Epoch 7/20
373/373 - 1s - loss: 9.2113e-04 - 1s/epoch - 3ms/step
Epoch 8/20
373/373 - 1s - loss: 7.7050e-04 - 1s/epoch - 3ms/step
Epoch 9/20
373/373 - 1s - loss: 6.7792e-04 - 1s/epoch - 3ms/step
Epoch 10/20
373/373 - 1s - loss: 5.7637e-04 - 1s/epoch - 3ms/step
Epoch 11/20
373/373 - 1s - loss: 5.4197e-04 - 1s/epoch - 3ms/step
Epoch 12/20
373/373 - 1s - loss: 4.9043e-04 - 1s/epoch - 4ms/step
Epoch 13/20
373/373 - 1s - loss: 4.7569e-04 - 1s/epoch - 3ms/step
Epoch 14/20
373/373 - 1s - loss: 4.3614e-04 - 1s/epoch - 4ms/step
Epoch 15/20
373/373 - 1s - loss: 4.3892e-04 - 1s/epoch - 4ms/step
Epoch 16/20
373/373 - 1s - loss: 4.1777e-0

In [26]:
y_pred= lstm.predict(x_test)

12/12 [==============================] - 1s 3ms/step


In [27]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 4.733573089431274e-05


In [28]:
lstm.save("hack.h5")

C:\Users\Priyanshul\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
new_df = pd.read_csv(r"C:\Users\Priyanshul\OneDrive\Desktop\Hackathon\stock-market-prediction-and-sentimental-analysis\Test_DJIA_Table.csv")
new_df=new_df.iloc[::-1]
# 2. Preprocess the features
# Assuming you have already defined the StandardScaler 'sc' for scaling features
new_features_scaled = sc.transform(new_df[['Open', 'High', 'Low']].values)

# 3. Create input sequences
x_new, _ = lstm_split(new_features_scaled, n_steps=2)

# 4. Make predictions
predictions = lstm.predict(x_new)

63/63 [==============================] - 0s 3ms/step


In [43]:
predictions

array([[0.48408136],
       [0.49460334],
       [0.49097195],
       ...,
       [0.9177067 ],
       [0.9409689 ],
       [0.96201813]], dtype=float32)

In [44]:
a=predictions*100000

In [45]:
a=predictions.flatten()
a=np.append(a,0)

In [47]:
#submission_df = pd.DataFrame({'Id': new_df['Date'], 'Close': a})
#submission_df.to_csv(r"C:\Users\Priyanshul\OneDrive\Desktop\Hackathon\stock-market-prediction-and-sentimental-analysis\asd.csv")

In [32]:
y_pred

array([[0.88309705],
       [0.8892064 ],
       [0.8938135 ],
       [0.89739335],
       [0.8932009 ],
       [0.892169  ],
       [0.89515054],
       [0.89634836],
       [0.8977643 ],
       [0.8961526 ],
       [0.8899971 ],
       [0.88620985],
       [0.88439465],
       [0.8794558 ],
       [0.8634107 ],
       [0.84795475],
       [0.8482077 ],
       [0.8448125 ],
       [0.8404769 ],
       [0.83789194],
       [0.8431423 ],
       [0.85100055],
       [0.8514216 ],
       [0.85553944],
       [0.8618468 ],
       [0.8618814 ],
       [0.86766064],
       [0.87798727],
       [0.8842653 ],
       [0.88979316],
       [0.8908186 ],
       [0.89284194],
       [0.89709055],
       [0.8987638 ],
       [0.8974527 ],
       [0.89662254],
       [0.8958112 ],
       [0.8953171 ],
       [0.89517784],
       [0.89747787],
       [0.89922106],
       [0.89399076],
       [0.89199805],
       [0.89309025],
       [0.8899982 ],
       [0.8894839 ],
       [0.89584374],
       [0.901